# PLANT TRAITS 2024 - Exploración de datos

En este primer script lo que se pretende realizar es una exploración de los datos disponibles para que, de acuerdo a lo investigado anteriormente, podamos asegurar que contamos con data precisa y óptima para el comienzo del proyecto. Para poder realizar esto, intentaré responder las preguntas que se plantean como las mejores para el análisis de la data disponible. Las preguntas son las siguientes:

1. Haz el conteo del número de registros que tienes - ¿es lo que esperabas?
2. Busca los valores nulos - ¿Cómo lidiar con ellos?
3. Verifica el tipo de distribución de cada columna - ¿Éstos hacen match con la distribución esperada?
4. Busca outliers - ¿Hay anomalias en la data, todos los valores son válidos?
5. Valida si tu data está balanceada - ¿Hay diferentes grupos que representan todo el conjunto de datos, hay suficientes ejemplos de cada uno de estos grupos
6. ¿Existen sesgos en tu data? -¿Hay subgrupos que tratas de manera más favorable que otros?
7. Para valores NA o NULL, en vez de eliminar aquellos registros que los contengan, mejor es sustituirlos por un valor apropiado: por ejemplo, para variables continuas, se puede utilizar la moda, media o mediana; para variables categóricas, se puede utilizar la moda o una nueva categoría (por ejemplo “NA”)
8. Verificar si el modelo es robusto o sensible a outliers

A partir de los resultados obtenidos de responder las preguntas es que daré un juicio final sobre los datos.

## Pregunta 1 - Conteo de registros

La pregunta en específico que se pretende responder en este punto es la siguiente: **_Haz el conteo del número de registros que tienes - ¿es lo que esperabas?_**

De primera instancia, podemos decir que se tienen dos tipos de datos disponibles: archivos CSV e imágenes en formato JPEG. Las imágenes se encuentran en dos carpetas, uno de entrenamiento y otro de prueba, la carpeta de entrenamiento contiene un total de 55,523 imágenes, mientras que la carpeta de prueba contiene un total de 13,907 imágenes, con esto, contamos con un **total de 69,430 imágenes de plantas en formato JPEG**.

Aquí no se va a realizar el conteo mediante alguna rutina debido a que el número de imágenes es muy grande y la memoria de este libro de Jupyter no alcanza para poder procesar todas las imágenes en una sola rutina de conteo. Para el conteo de registros en los archivos CSV se realizará lo siguiente:

In [8]:
#Importamos las librerías a utilizar
import pandas as pd
import functions.general_functions as gf

#Obtenemos la ruta de los archivos CSV
ruta_archivos = gf.get_data_path('csv')
nombre_archivo = 'train.csv'
nombre_archivo_t = 'test.csv'

#Leemos el archivo csv
df_train = pd.read_csv(ruta_archivos + nombre_archivo)
df_test = pd.read_csv(ruta_archivos + nombre_archivo_t)

#Obtenemos el número de registros para cada tabla
num_reg_train = str(df_train.shape[0])
num_reg_test = str(df_test.shape[0])
num_reg_total = str(df_train.shape[0] + df_test.shape[0])

#Imprimimos los totales
print('El conjunto/tabla de entrenamiento contiene un total de ' + num_reg_train + ' registros')
print('El conjunto/tabla de prueba contiene un total de ' + num_reg_test + ' registros')
print('En total, se cuenta con ' + num_reg_total + ' registros')

El conjunto/tabla de entrenamiento contiene un total de 55489 registros
El conjunto/tabla de prueba contiene un total de 13873 registros
En total, se cuenta con 69362 registros


De lo anterior, podemos obtener las siguientes conclusiones:

1. Para el conjunto de entrenamiento, se tienen 55,523 imágenes y 55,489 registros en su archivo CSV por lo que sugiere que para algunas fotos (34) no se tiene su respectiva información dentro del archivo CSV
2. Para el conjunto de prueba, se tienen 13,907 imágenes y 13,873 registros en su archivo CSV por lo que sugiere que para algunas fotos (34) no se tiene su respectiva información dentro del archivo CSV
3. En total, se tiene un total de 68 registros menos en CSV que en imágenes.

De manera general, podemos decir que **el volúmen de los datos si es lo esperado**, es un volúmen grande que nos ayudará a hacer un mejor entrenamiento del modelo. El número de registros faltantes en verdad es muy poco (0.001%) comparado con el total de registros que si se tienen, por lo que no representa una amenaza para el proyecto. En este punto la data se encuentra perfecto.

## Pregunta 2 - Valores nulos

La pregunta en específico que se pretende responder en este punto es la siguiente: **_Busca los valores nulos - ¿Cómo lidiar con ellos?_**

De primera instancia, para el conjunto de imágenes sabemos que no existen valores nulos ya que son imágenes y no existe algo como "imagen nula", aquí simplemente se revisa si se tiene la imagen o no. Por otro lado, dentro de los archivos CSV si pueden llegar a existir valores nulos, por lo que se realizará la investigación pertinente.

In [57]:
#Se pretende desplegar una tabla final con el nombre de las columnas, el tipo de cada columna y el número de sus registros nulos, 
#esto para cada conjunto (entrenamiento y prueba)

#Obtenemos los nombres de las columnas en una lista para cada conjunto de datos
col_train = list(df_train.columns)
col_test = list(df_test.columns)

#Proceso para obtener el número de registros null y el tipo de dato para cada columna
#Creamos listas nulas para ir guardando los números
num_nulls_train = []
num_nulls_test = []
type_train = []
type_test = []
#Hacemos el loop para ir guardando los números del conj de entrenamiento
for x in col_train:
    #Obtenemos el número de registros null
    num_nulls = df_train[x].isnull().sum()
    #Guardamos el número en la lista
    num_nulls_train.append(num_nulls)
    #Obtenemos el tipo de dato en la columna
    type_data = str(df_train[x].dtypes)
    #Guardamos el tipo de dato en la lista
    type_train.append(type_data)
#Hacemos el loop para ir guardando los números del conj de prueba
for y in col_test:
    #Obtenemos el número de registros null
    num_nulls = df_test[y].isnull().sum()
    #Guardamos el número en la lista
    num_nulls_test.append(num_nulls)
    #Obtenemos el tipo de dato en la columna
    type_data = str(df_test[y].dtypes)
    #Guardamos el tipo de dato en la lista
    type_test.append(type_data)

#Creamos lista que denota el tipo de conjunto de datos
tipo_train = ['train'] * len(col_train)
tipo_test = ['test'] * len(col_test)

#Creamos el diccionario para el conj de entrenamiento para después convertirlo en df
dic_train = {'type':tipo_train,'columns':col_train,'data_type':type_train,'num_nulls':num_nulls_train}
#Creamos el diccionario para el conj de prueba para después convertirlo en df
dic_test = {'type':tipo_test,'columns':col_test,'data_type':type_test,'num_nulls':num_nulls_test}

#Creamos el df para el conj de entrenamiento
df_col_train = pd.DataFrame(dic_train)
#Creamos el df para el conj de prueba
df_col_test = pd.DataFrame(dic_test)

#Unimos ambos dfs
df_cols = pd.concat([df_col_train,df_col_test], ignore_index = True)

#Obtenemos aquellos registros con num_nulls > 0
df_cols[df_cols['num_nulls'] > 0]

,type,columns,data_type,num_nulls
170,train,X4_sd,float64,16341
171,train,X11_sd,float64,16341
172,train,X18_sd,float64,16341
173,train,X26_sd,float64,16341
174,train,X50_sd,float64,16341
175,train,X3112_sd,float64,16341


Podemos ver que existen 6 columnas que tienen algunos registros nulos. Según los tips, podemos sustituir por valores promedio. **CONFIRMAR ESTA PARTE**

In [58]:
df_train[df_train['X4_sd'].isnull()]

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,X18_mean,X26_mean,X50_mean,X3112_mean,X4_sd,X11_sd,X18_sd,X26_sd,X50_sd,X3112_sd
2,196639184,14.254504,902.071411,49.642857,17.873655,387.977753,22.807142,107,133,119,...,8.552908,0.395241,2.343153,1130.096731,NaN,NaN,NaN,NaN,NaN,NaN
6,193133748,0.271833,768.048889,50.986668,27.305773,1299.451904,49.694664,101,160,139,...,0.906575,1.368527,1.195674,709.163496,NaN,NaN,NaN,NaN,NaN,NaN
7,195910523,13.004167,1026.000000,53.000000,14.918256,379.825012,20.049999,99,117,104,...,6.001135,3.566168,1.733709,2062.304630,NaN,NaN,NaN,NaN,NaN,NaN
12,196314343,9.946926,238.151108,14.048889,18.266577,833.824524,40.026669,145,153,148,...,1.471547,1.254147,1.734942,414.795507,NaN,NaN,NaN,NaN,NaN,NaN
18,176956365,22.926111,3223.385742,393.752380,57.237907,47.252399,9.375714,103,116,107,...,5.807784,3844.356802,2.015860,27076.214230,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55479,143375475,25.342134,1269.821411,273.744904,104.328812,91.052284,14.505612,124,137,130,...,0.922993,3.949954,1.294756,3123.383076,NaN,NaN,NaN,NaN,NaN,NaN
55481,193145264,15.907291,1166.500000,184.500000,72.396286,280.652466,20.049999,119,128,122,...,0.391445,0.950440,1.648881,406.806396,NaN,NaN,NaN,NaN,NaN,NaN
55483,194246645,10.237585,746.724487,38.000000,17.433586,499.778442,26.671429,108,136,122,...,0.431378,2.592842,1.913140,279.204596,NaN,NaN,NaN,NaN,NaN,NaN
55484,190558785,19.472172,244.795914,39.127552,67.074493,472.710358,27.758673,118,140,131,...,0.233690,1.783193,1.608341,969.547831,NaN,NaN,NaN,NaN,NaN,NaN
